**融合 CUDA 内核**

当在 GPU 上运行计算时，所需数据会从内存中获取，然后运行计算并将结果保存回内存。简单来说，融合内核的概念是将通常由 Pytorch 分别执行的操作合并为单个硬件操作。因此，通过将多个离散计算合并为一个操作，它们减少了内存移动次数。

<div align="center">
<img src="https://huggingface.co/blog/assets/100_megatron_training//kernel_fusion.png" alt="fused kernel" width="40%">
</div>

当 f、g 和 h 在一个内核中融合时，f 和 g 的中介结果 x' 和 y'会存储在 GPU 寄存器中，并由 h 立即使用。但没有融合的情况下，x'和 y'需要复制到内存中，然后由 h 加载。因此，融合内核显著提高了计算速度。

Megatron-LM 也使用 Apex 的 AdamW 融合实现，该实现比 Pytorch 实现更快。*虽然可以像 Megatron-LM 一样自定义 DataLoader，并使用 transformers 的 Apex 融合优化器，但构建自定义融合 CUDA 内核对初学者并不友好。

**Apex**

Apex 是 NVIDIA 维护的 PyTorch 扩展库，用来加速训练，主要提供两类东西——混合精度训练工具和一批融合（fused）的 CUDA 算子/优化器（比如 FusedAdam / FusedAdamW），以减少内核启动与显存读写开销、提升吞吐。很多大模型训练框架（如 Megatron-LM）都会用到它的 fused 优化器来提速。

- Apex 里的 FusedAdam/FusedAdamW 把 Adam/AdamW 的逐元素更新步骤和多张量批处理合并到更少的 CUDA 内核里执行，可作为 torch.optim 的即插即用替代以获得更高性能；
- Apex 的旧版 apex.amp（混合精度 API）已被官方标注为弃用，推荐改用 PyTorch 原生的 torch.amp / torch.cuda.amp；近年的 PyTorch 也给自带优化器加了 `fused=True/foreach=True` 等实现（如 AdamW(fused=True)），在不少场景能获得与 Apex 类似的加速，这也是为什么很多项目正在减少对 Apex 的依赖；

**Mixed Precision**

混合精度（Mixed Precision, MP）是在一次训练 / 推理里把不同算子用不同数值精度来计算：对矩阵乘、卷积这类在低精度更快的算子用 `FP16/BF16`，对归约、软最大值这类更敏感的算子仍用 `FP32`；从而同时获得速度提升与显存占用下降。在 PyTorch 里这就是 torch.amp/torch.cuda.amp 做的事。

- 权重、激活和梯度用低精度可把显存占用近似减半，常见做法是保留 FP32 的“主权重 / 累加”副本来做优化步，前反 / 反传用半精度；
- BF16：对 LLM 这类深层 Transformer，BF16 因为动态范围和 FP32 一样大（8 位指数），通常较 FP16 更稳定、基本不需要做“损失缩放（loss scaling）”；Hugging Face 的文档也建议 Ampere 及更新架构优先用 BF16；
- 损失缩放（Loss Scaling）：FP16 容易梯度下溢，Micikevicius 等提出先把 loss 乘一个系数再反传，并在检测到溢出时动态调整与跳过更新；PyTorch 的 GradScaler 就是该做法的实现；
- 在 PyTorch 用 autocast 指定低 / 高精度的算子选择，配合 GradScaler（若用 FP16）；
- LLM 推理常用 FP16 / BF16 混合或“全 BF16”以增加吞吐并降低显存占用（例如 Transformers 的推理开关）；
- 迈向 FP8（更激进的混合精度）：在 H100 / H200 等硬件上，NVIDIA 的 Transformer Engine 把 Transformer 模块的部分计算降到 FP8，并管理比例因子、缩放以保持稳定；近期像 DeepL 把 LLM 训练从 BF16 迁到 FP8 就是用的这一套；

**梯度下溢**

下溢就是数值太小，小到当前浮点格式已经表示不了，会被舍入成 0（或成为更粗糙的次正规数/更少有效位数）。在 IEEE-754 里，下溢本质上是“结果太小而无法在目标格式正常表示”，一般会导致精度丢失或直接变成 0。

在深度学习里，若用 FP16 训练，很小的梯度容易在反传时“掉到 0”，导致学习停滞，于是才有了 Loss Scaling（损失缩放）把梯度整体放大以后再回缩，用以减少梯度下溢。

- FP16（binary16）：最大值 65504；最小正常数约 6.10×10⁻⁵；最小次正规正数 subnormal 约 5.96×10⁻⁸（再小就变 0）；
- BF16：和 FP32 一样 8 位指数，动态范围与 FP32 近似相同，但只有 7 位尾数（精度低于 FP32 / FP16）；

如下，float16：b 可能直接变成 0.0e+00，从而 a*b = 0（下溢）。这与 FP16 的最小次正规数约 5.96e-8 一致。

In [1]:
import torch

def tiny_mul(dtype):
    a = torch.tensor(1.0, dtype=dtype)
    b = torch.tensor(1e-8, dtype=dtype)  # 1e-8 < FP16 minimum subnormal ~5.96e-8
    c = a * b
    return a, b, c

for dt in [torch.float16, torch.bfloat16, torch.float32]:
    a, b, c = tiny_mul(dt)
    print(f"{dt}: a={a.item():.1e}, b={b.item():.1e}, a*b={c.item():.1e}")

torch.float16: a=1.0e+00, b=0.0e+00, a*b=0.0e+00
torch.bfloat16: a=1.0e+00, b=1.0e-08, a*b=1.0e-08
torch.float32: a=1.0e+00, b=1.0e-08, a*b=1.0e-08


如下，FP16 平均梯度接近 0（很多情况下直接为 0），体现梯度下溢。这正是 GradScaler 要解决的问题：先把 loss 放大，再把梯度 / 更新回缩。

In [12]:
import torch

device = "mps"

def grad_once(dtype):
    w = torch.ones(1024, device=device, dtype=dtype, requires_grad=True)
    scale = torch.tensor(1e-8, device=device, dtype=dtype)  # underflow in fp16
    y = w.sum() * scale
    y.backward()
    return y.detach().float().item(), w.grad.float().mean().item()

for dt in [torch.float16, torch.float32]:
    loss, gmean = grad_once(dt)
    print(dt, "loss=", loss, "mean_grad=", gmean)

torch.float16 loss= 0.0 mean_grad= 0.0
torch.float32 loss= 1.0239999937766697e-05 mean_grad= 9.99999993922529e-09


- float16 / bfloat16 / float32 / float64 分别占 2 / 2 / 4 / 8 字节；
- finfo.tiny 会看到 float16 的最小正常数 ≈ 6.10e-05，而 bfloat16 的最小正常数 ≈ 1.17e-38，与 FP32 同级（因此范围更大）；

In [ ]:
import torch

def info(dtype):
    x = torch.empty(1_000_000, dtype=dtype)
    finfo = torch.finfo(dtype)
    return {
        "dtype": str(dtype),
        "bits_per_elem": x.element_size() * 8,
        "bytes_per_elem": x.element_size(),
        "approx_mem_for_1M(MB)": x.numel() * x.element_size() / 1e6,
        "min_normal(tiny)": float(finfo.tiny),
        "max": float(finfo.max),
        "eps": float(finfo.eps),
    }

rows = [info(dt) for dt in [torch.float16, torch.bfloat16, torch.float32, torch.float64]]
for r in rows:
    print(r)

{'dtype': 'torch.float16', 'bits_per_elem': 16, 'bytes_per_elem': 2, 'approx_mem_for_1M(MB)': 2.0, 'min_normal(tiny)': 6.103515625e-05, 'max': 65504.0, 'eps': 0.0009765625}
{'dtype': 'torch.bfloat16', 'bits_per_elem': 16, 'bytes_per_elem': 2, 'approx_mem_for_1M(MB)': 2.0, 'min_normal(tiny)': 1.1754943508222875e-38, 'max': 3.3895313892515355e+38, 'eps': 0.0078125}
{'dtype': 'torch.float32', 'bits_per_elem': 32, 'bytes_per_elem': 4, 'approx_mem_for_1M(MB)': 4.0, 'min_normal(tiny)': 1.1754943508222875e-38, 'max': 3.4028234663852886e+38, 'eps': 1.1920928955078125e-07}
{'dtype': 'torch.float64', 'bits_per_elem': 64, 'bytes_per_elem': 8, 'approx_mem_for_1M(MB)': 8.0, 'min_normal(tiny)': 2.2250738585072014e-308, 'max': 1.7976931348623157e+308, 'eps': 2.220446049250313e-16}


In [17]:
torch.empty(1_000_000, dtype=torch.float32).element_size()

4

在 PyTorch 里用 AMP（torch.amp / torch.cuda.amp）的使用。autocast 负责自动挑选低 / 高精度算子。首先是训练（FP16，带 GradScaler），减少梯度下溢。

In [20]:
import torch, torch.nn as nn

model = nn.Linear(4096, 4096).to('mps')
optim = torch.optim.AdamW(model.parameters(), lr=1e-3)

scaler = torch.amp.GradScaler()
for step in range(100):
    x = torch.randn(32, 4096, device='mps')
    y = torch.randn(32, 4096, device='mps')

    optim.zero_grad(set_to_none=True)
    with torch.amp.autocast(device_type='mps', dtype=torch.float16):
        loss = (model(x) - y).pow(2).mean()

    scaler.scale(loss).backward()
    scaler.step(optim)
    scaler.update()

print(f"step {step} loss={loss.item()}")

step 99 loss=1.399855613708496


BF16：更大的动态范围，一般**不需要** GradScaler。

In [21]:
import torch, torch.nn as nn

model = nn.Linear(4096, 4096).to('mps')
optim = torch.optim.AdamW(model.parameters(), lr=1e-3)

for step in range(100):
    x = torch.randn(32, 4096, device='mps')
    y = torch.randn(32, 4096, device='mps')

    optim.zero_grad(set_to_none=True)
    with torch.amp.autocast(device_type='mps', dtype=torch.bfloat16):
        loss = (model(x) - y).pow(2).mean()

    loss.backward()
    optim.step()

print(f"step {step} loss={loss.item()}")

step 99 loss=1.4118238687515259


PyTorch 的 fused 优化器主要是 CUDA 路径上的实现；AdamW 文档里还专门提到 MPS 目前只是“prototype 实现，支持 FP32 / FP16”（没有宣传 fused）。在 MPS 上把 fused=True 传进去通常不会带来期望的 CUDA fused 核心收益。

结果并不显著，因为首先 fused/foreach 优势通常在参数很多、每个张量较小时更明显，减少**按张量循环 + 多次内核启动**的开销；而单个大层（比如 4096×4096 的几层）效果不一定显著。

In [28]:
import time, copy
import torch, torch.nn as nn

assert torch.backends.mps.is_available()
device = "mps"
dtype = torch.float16
torch.manual_seed(0)

def make_model():
    m = nn.Sequential(nn.Linear(4096, 4096), nn.ReLU(), nn.Linear(4096, 4096))
    return m.to(device).to(dtype)

def bench(optimizer_ctor, steps=100, warmup=10):
    model = make_model()
    opt = optimizer_ctor(model.parameters())
    torch.mps.synchronize()

    # warmup
    for _ in range(warmup):
        x = torch.randn(32, 4096, device=device, dtype=dtype)
        y = torch.randn(32, 4096, device=device, dtype=dtype)
        opt.zero_grad(set_to_none=True)
        loss = (model(x) - y).pow(2).mean()
        loss.backward()
        opt.step()

    # measure optimizer.step()
    total = 0.0
    for _ in range(steps):
        x = torch.randn(32, 4096, device=device, dtype=dtype)
        y = torch.randn(32, 4096, device=device, dtype=dtype)
        opt.zero_grad(set_to_none=True)
        with torch.amp.autocast(device_type="mps", dtype=dtype):
            loss = (model(x) - y).pow(2).mean()
        loss.backward()
        torch.mps.synchronize()
        t0 = time.perf_counter()
        opt.step()
        torch.mps.synchronize()
        total += time.perf_counter() - t0
    return total

# compare foreach=False vs foreach=True
base_state = make_model().state_dict()

def ctor_foreach(flag):
    def _ctor(params):
        opt = torch.optim.AdamW(params, lr=1e-3, foreach=flag)
        return opt
    return _ctor

# run (reset the model to the same initial weight for each run)
mA = make_model(); mA.load_state_dict(base_state)
tA = bench(lambda p: torch.optim.AdamW(p, lr=1e-3, foreach=False))

mB = make_model(); mB.load_state_dict(base_state)
tB = bench(lambda p: torch.optim.AdamW(p, lr=1e-3, foreach=True))

print(f"AdamW foreach=False, step-time sum: {tA:.4f}s")
print(f"AdamW foreach=True,  step-time sum: {tB:.4f}s")

AdamW foreach=False, step-time sum: 0.6074s
AdamW foreach=True,  step-time sum: 0.6198s
